In [ ]:
#Installation des biblothéques nécessaires
!pip install transformers
!pip install torch
!pip install streamlit
!pip install rouge-score
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=822ca1d9e4fef340a7e93ff1fad53ab5e6875bdd54b25a58517ef458c15cc9e6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
#Chargement des modéles GPT2 et GPT2 Neo
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM

# Load GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Load GPT-Neo model and tokenizer
gpt_neo_tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_neo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [ ]:
#Generation des poéme pour les 2 modéles
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "Donald Trump - Let's make America great again !!!"
gpt2_poem = generate_text(prompt, gpt2_model, gpt2_tokenizer)
gpt_neo_poem = generate_text(prompt, gpt_neo_model, gpt_neo_tokenizer)

print("GPT-2 Poem:\n", gpt2_poem)
print("GPT-Neo Poem:\n", gpt_neo_poem)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Poem:
 Donald Trump - Let's make America great again !!!!" The tweet is the most comprehensive record I've seen from any presidential candidate since 1988.

- Let's make America great again !!!!" The tweet is the most comprehensive record I've
GPT-Neo Poem:
 Donald Trump - Let's make America great again !!!

How do you get a big hit when the whole world is watching? When the US is the most watched and trusted global media and media personality? When the President can create news that does


In [ ]:
#Calcule des scores bleu et rouge
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

def compute_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    return sentence_bleu(reference, candidate)

def compute_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    return scorer.score(reference, candidate)

# Example usage:
bleu_score = compute_bleu(gpt2_poem, gpt_neo_poem)
rouge_scores = compute_rouge(gpt2_poem, gpt_neo_poem)

print("BLEU Score:", bleu_score)
print("ROUGE Scores:", rouge_scores)

BLEU Score: 0.18828898171079816
ROUGE Scores: {'rouge1': Score(precision=0.3488372093023256, recall=0.38461538461538464, fmeasure=0.3658536585365854), 'rouge2': Score(precision=0.21428571428571427, recall=0.23684210526315788, fmeasure=0.225), 'rougeL': Score(precision=0.32558139534883723, recall=0.358974358974359, fmeasure=0.34146341463414637)}


Le score Bleu nous montre que on à 18% de coherence des mots génere par le poéme avec les mots clé fourni.
Le score rouge nous montre une correspondance d'environ 0.2 pour chaque mot individu, correspondance de 0.16 pour sequence de 2 mots, et correspondance de 0.2 pour plus plusiers mots.

In [ ]:
!pip install pyngrok

In [ ]:
%%writefile app.py
#Création d'application
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Chargement des modeles
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt_neo_tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt_neo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

#Fonction pour genere le poem (limite à 50 mots pour des raisons de chargement liée a CPU)
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

#Calcule des score bleu et rouge
def compute_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    return sentence_bleu(reference, candidate)

def compute_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    return scorer.score(reference, candidate)

# Création d'interface
st.title("Poem Generator with GPT-2 and GPT-Neo")
prompt = st.text_input("Enter a keyword or phrase:", "Let's make America great again !!!")

if st.button("Generate Poem"):
    gpt2_poem = generate_text(prompt, gpt2_model, gpt2_tokenizer)
    gpt_neo_poem = generate_text(prompt, gpt_neo_model, gpt_neo_tokenizer)

    st.subheader("Generated Poem (GPT-2):")
    st.text(gpt2_poem)

    st.subheader("Generated Poem (GPT-Neo):")
    st.text(gpt_neo_poem)

    # Compute scores
    bleu_score = compute_bleu(gpt2_poem, gpt_neo_poem)
    rouge_scores = compute_rouge(gpt2_poem, gpt_neo_poem)

    st.subheader("Evaluation Metrics:")
    st.text(f"BLEU Score: {bleu_score}")
    st.text(f"ROUGE Scores: {rouge_scores}")

Writing app.py


In [ ]:
#Veuillez crée une compte sur ngrok pour obtenir une authtoken
!ngrok config add-authtoken 2sB9NH2jIyArO0WYrFAFO7kIQCv_3qdNw1RebiAxMrCh7FKwz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#Création d'une tunnel pour visiter le site avec l'interface
from pyngrok import ngrok
import threading
import os

# Fonction pour démarée Streamlit app
def run_streamlit():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run_streamlit)
thread.start()

# Démarée le tunnel ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit app is available at: {public_url}")

Streamlit app is available at: NgrokTunnel: "https://bd25-34-83-91-58.ngrok-free.app" -> "http://localhost:8501"
